In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import mediapipe as mp
import arabic_reshaper
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from PIL import ImageFont, ImageDraw, Image
import arabic_reshaper
from bidi.algorithm import get_display

In [ ]:
import cv2
import os
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

folder_path = r"D:\arabic_datases_cnn\ARSL-No-BackGround-V2"
save_path = r"D:\arabic_datases_cnn\mediapipe"

for dir_ in os.listdir(folder_path):
    dir_path = os.path.join(folder_path, dir_)
    save_dir = os.path.join(save_path, dir_)
    os.makedirs(save_dir, exist_ok=True)

    for img_name in os.listdir(dir_path):
        img_path = os.path.join(dir_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb = cv2.resize(img_rgb, (224, 224))
        img_rgb = cv2.flip(img_rgb, 1)

        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
                save_img_path = os.path.join(save_dir, img_name)
                cv2.imwrite(save_img_path, cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

In [23]:
import random

folder_path_2 = r"D:\arabic_datases_cnn\mediapipe - Copy (2)"


target = 500

for class_name in os.listdir(folder_path_2):
    class_dir = os.path.join(folder_path_2, class_name)
    if not os.path.isdir(class_dir):
        continue
        
    images = []
    for img_name in os.listdir(class_dir):
        if img_name.endswith(('.jpg', '.jpeg', '.png')):
            images.append(os.path.join(class_dir, img_name))
    if len(images) > target:
        images_to_remove = random.sample(images, len(images) - target)
        for img_path in images_to_remove:
            os.remove(img_path)
            print(f"Removed {img_path}")

Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_667_Ain_48.jpeg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_912_aug_1_Ain_0.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_107_Ain_129.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_439_Ain_21.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_304_Ain_18.jpeg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_216_Ain_156.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_16_Ain_101.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_205_Ain_154.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_199_Ain_152.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\Ain_57.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_736_Ain_65.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_885_aug_15_Ain_101.jpg
Removed D:\arabic_datases_cnn\mediapipe - Copy (2)\Ain\aug_257_Ain_165.jpg
Removed D:\arabic_datases_

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

img_size = (224, 224)  
data_dir = r"D:\arabic_datases_cnn\mediapipe - Copy"

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    zoom_range=0.05,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05
)
val_split = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle =True
)

val_data = val_split.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle =False
)

Found 7501 images belonging to 31 classes.
Found 1866 images belonging to 31 classes.


In [ ]:
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
# from tensorflow.keras.optimizers import Adam

# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# base_model.trainable = False  

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.3)(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.3)(x)

# predictions = Dense(train_data.num_classes, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# تحميل MobileNetV2 بدون الطبقة العلوية الأصلية (1000 class head)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# تجميد الطبقات الأساسية (اختياري في البداية للتدريب السريع)
base_model.trainable = False

# بناء طبقة الإخراج الجديدة المناسبة لعدد الأصناف الجديدة
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

# بناء النموذج النهائي
model = Model(inputs=base_model.input, outputs=predictions)

# تجميع النموذج (compile)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[early_stop, reduce_lr, checkpoint])

c:\Users\Samer\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 520ms/step - accuracy: 0.0807 - loss: 3.3688

469/469 ━━━━━━━━━━━━━━━━━━━━ 299s 626ms/step - accuracy: 0.0809 - loss: 3.3683 - val_accuracy: 0.3966 - val_loss: 2.3431 - learning_rate: 1.0000e-04
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 524ms/step - accuracy: 0.2914 - loss: 2.4239

469/469 ━━━━━━━━━━━━━━━━━━━━ 294s 627ms/step - accuracy: 0.2915 - loss: 2.4235 - val_accuracy: 0.5927 - val_loss: 1.5984 - learning_rate: 1.0000e-04
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 522ms/step - accuracy: 0.4208 - loss: 1.8690

469/469 ━━━━━━━━━━━━━━━━━━━━ 290s 618ms/step - accuracy: 0.4209 - loss: 1.8688 - val_accuracy: 0.6688 - val_loss: 1.2694 - learning_rate: 1.0000e-04
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step - accuracy: 0.5076 - loss: 1.5458

...

469/469 ━━━━━━━━━━━━━━━━━━━━ 255s 543ms/step - accuracy: 0.8907 - loss: 0.3101 - val_accuracy: 0.8917 - val_loss: 0.3308 - learning_rate: 1.0000e-04
Epoch 35/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 261s 556ms/step - accuracy: 0.9060 - loss: 0.2858 - val_accuracy: 0.8869 - val_loss: 0.3431 - learning_rate: 1.0000e-04
Epoch 36/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 259s 552ms/step - accuracy: 0.8904 - loss: 0.3007 - val_accuracy: 0.8880 - val_loss: 0.3324 - learning_rate: 1.0000e-04
Epoch 37/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 251s 535ms/step - accuracy: 0.9187 - loss: 0.2670 - val_accuracy: 0.8896 - val_loss: 0.3307 - learning_rate: 1.0000e-04
Epoch 38/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step - accuracy: 0.8997 - loss: 0.2852

469/469 ━━━━━━━━━━━━━━━━━━━━ 262s 558ms/step - accuracy: 0.8997 - loss: 0.2852 - val_accuracy: 0.9009 - val_loss: 0.3188 - learning_rate: 1.0000e-04
Epoch 39/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 243s 519ms/step - accuracy: 0.9044 - loss: 0.2826 - val_accuracy: 0.8875 - val_loss: 0.3438 - learning_rate: 1.0000e-04
Epoch 40/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 245s 523ms/step - accuracy: 0.9071 - loss: 0.2719 - val_accuracy: 0.8816 - val_loss: 0.3498 - learning_rate: 1.0000e-04
Epoch 41/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 246s 525ms/step - accuracy: 0.9211 - loss: 0.2450 - val_accuracy: 0.8982 - val_loss: 0.3215 - learning_rate: 1.0000e-04
Epoch 42/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 246s 524ms/step - accuracy: 0.9234 - loss: 0.2375 - val_accuracy: 0.8976 - val_loss: 0.3266 - learning_rate: 1.0000e-04
Epoch 43/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 232s 495ms/step - accuracy: 0.9217 - loss: 0.2442 - val_accuracy: 0.8987 - val_loss: 0.3209 - learning_rate: 1.0000e-04
Epoch 44/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 40

469/469 ━━━━━━━━━━━━━━━━━━━━ 227s 484ms/step - accuracy: 0.9308 - loss: 0.2127 - val_accuracy: 0.9046 - val_loss: 0.3107 - learning_rate: 2.0000e-05
Epoch 45/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 226s 482ms/step - accuracy: 0.9400 - loss: 0.1903 - val_accuracy: 0.9030 - val_loss: 0.3076 - learning_rate: 2.0000e-05
Epoch 46/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step - accuracy: 0.9264 - loss: 0.2062

469/469 ━━━━━━━━━━━━━━━━━━━━ 227s 484ms/step - accuracy: 0.9264 - loss: 0.2062 - val_accuracy: 0.9051 - val_loss: 0.3018 - learning_rate: 2.0000e-05
Epoch 47/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 227s 483ms/step - accuracy: 0.9349 - loss: 0.2067 - val_accuracy: 0.9030 - val_loss: 0.3017 - learning_rate: 2.0000e-05
Epoch 48/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 227s 485ms/step - accuracy: 0.9371 - loss: 0.2031 - val_accuracy: 0.9035 - val_loss: 0.3005 - learning_rate: 2.0000e-05
Epoch 49/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 226s 481ms/step - accuracy: 0.9307 - loss: 0.2064 - val_accuracy: 0.9041 - val_loss: 0.3067 - learning_rate: 2.0000e-05
Epoch 50/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step - accuracy: 0.9334 - loss: 0.1970

469/469 ━━━━━━━━━━━━━━━━━━━━ 227s 483ms/step - accuracy: 0.9334 - loss: 0.1970 - val_accuracy: 0.9057 - val_loss: 0.3047 - learning_rate: 2.0000e-05


In [14]:
# for layer in base_model.layers[-30:]:
#     layer.trainable = True
    
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# model.fit(train_data, validation_data=val_data, epochs=100, callbacks=[early_stop, reduce_lr, checkpoint])

In [23]:
model.save('pre_90.h5')

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
# import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

val_steps = val_data.samples // val_data.batch_size + 1

predictions = model.predict(val_data, steps=val_steps)
y_pred = np.argmax(predictions, axis=1)
y_true = val_data.classes

class_names = list(val_data.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

# plt.figure(figsize=(16, 12))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
# plt.xlabel("Predicted")
# plt.ylabel("True")
# plt.title("Confusion Matrix")
# plt.xticks(rotation=90)
# plt.yticks(rotation=0)
# plt.tight_layout()
# plt.show()


117/117 ━━━━━━━━━━━━━━━━━━━━ 46s 382ms/step
Classification Report:

              precision    recall  f1-score   support

         Ain       0.92      0.95      0.93        60
          Al       0.94      0.98      0.96        60
        Alef       0.98      0.98      0.98        62
         Beh       0.90      0.85      0.87        61
         Dad       0.97      0.95      0.96        60
         Dal       0.88      0.85      0.86        60
         Feh       0.77      0.88      0.82        60
       Ghain       0.95      0.97      0.96        60
         Hah       0.95      0.95      0.95        60
         Heh       0.90      0.78      0.84        60
        Jeem       0.85      0.88      0.87        60
         Kaf       0.84      0.80      0.82        60
        Khah       0.95      0.98      0.97        60
         Laa       0.89      0.93      0.91        60
         Lam       0.90      0.95      0.93        60
        Meem       0.83      0.82      0.82        60
        Noon 

In [16]:
for i, name in enumerate(class_names):
    print(f"{i}: {name}")


0: Ain
1: Al
2: Alef
3: Beh
4: Dad
5: Dal
6: Feh
7: Ghain
8: Hah
9: Heh
10: Jeem
11: Kaf
12: Khah
13: Laa
14: Lam
15: Meem
16: Noon
17: Qaf
18: Reh
19: Sad
20: Seen
21: Sheen
22: Tah
23: Teh
24: Teh_Marbuta
25: Thal
26: Theh
27: Waw
28: Yeh
29: Zah
30: Zain


In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import os
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

model = load_model("pre_90.h5")
labels = sorted(os.listdir(r"D:\arabic_datases_cnn\mediapipe - Copy")) 

IMAGE_SIZE = 224

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip the frame horizontally
    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            x_ = []
            y_ = []
            data_aux = []

            for lm in hand_landmarks.landmark:
                x_.append(lm.x)
                y_.append(lm.y)

            for lm in hand_landmarks.landmark:
                data_aux.append(lm.x - min(x_))
                data_aux.append(lm.y - min(y_))

        x1 = int(min(x_) * W) - 20
        y1 = int(min(y_) * H) - 20
        x2 = int(max(x_) * W) + 20
        y2 = int(max(y_) * H) + 20

        roi = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
        roi = img_to_array(roi)
        roi = roi.astype("float32") / 255.0
        roi = np.expand_dims(roi, axis=0)

        pred = model.predict(roi)[0]
        index = np.argmax(pred)
        label = labels[index]
        confidence = pred[index]

        cv2.putText(frame, f'{label} ({confidence:.2f})', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.imshow("Real-Time Sign Language Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import os

model = load_model("pre_90.h5")
labels = sorted(os.listdir(r"D:\arabic_datases_cnn\mediapipe - Copy"))  # تأكد أن ترتيب المجلدات هو نفس ترتيب التدريب
IMAGE_SIZE = 224

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)  # Flip the frame horizontally

    roi = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
    roi = img_to_array(roi)
    roi = roi.astype("float32") / 255.0
    roi = np.expand_dims(roi, axis=0)

    pred = model.predict(roi)[0]
    index = np.argmax(pred)
    label = labels[index]
    confidence = pred[index]

    cv2.putText(frame, f'{label} ({confidence:.2f})', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow("Real-Time Sign Language Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━